In [3]:
import wandb
import pandas as pd
import numpy as np
import texttable as tt
import latextable

In [10]:
EXPERIMENT_GROUP = "exp-13-neck-reduction"
BASELINE_SETUP = "full-neck"
AVERAGE_EPOCHS = 0 # 5 or 0 | 0 for no averaging, must be odd
HAS_WEIGHTED_ACCURACY = True
FIRST_N_METRICS = 6 #6 # Only weighted_accuracy and macro_f1 for the results table in experiment results section
EXCLUDED_SETUPS = [0] # [0, 1] | list of setups to exclude, by their index in the list of setups

In [55]:
def get_best_epoch_metrics(run, metric_name, average_epochs=0):
    history = pd.DataFrame(list(run.scan_history()))
    
    # Find the epoch with the best metric value
    best_epoch = history[metric_name].idxmax()
    best_metrics = history.loc[best_epoch]
    if average_epochs > 0:
        # Find indices of valid (non-NaN) values
        valid_indices = history[metric_name].dropna().index
        
        # Locate the best epoch index in the valid indices
        best_index_position = valid_indices.get_loc(best_epoch)
        
        # Get the indices of the average_epochs epochs to be averaged
        idx_adjustment = (average_epochs - 1) // 2
        start_index = max(0, best_index_position - idx_adjustment)
        end_index = min(len(valid_indices), best_index_position + idx_adjustment + 1)
        selected_indices = valid_indices[start_index:end_index]
        
        # Convert the selected indices to numeric, forcing errors to NaN - Use when averaging over epchs
        # converted_history = history.loc[selected_indices].apply(pd.to_numeric, errors='coerce')
        # converted_history.mean()

        # Average the metrics over the selected epochs
        averaged_metrics = history.loc[selected_indices].mean()
        
        print(f"Run {run.url} - Best epoch: {best_epoch} (averaged over {average_epochs}) - {metric_name}: {averaged_metrics[metric_name]}")
        return averaged_metrics, best_epoch
    else:
        print(f"Run {run.url} - Best epoch: {best_epoch} - {metric_name}: {best_metrics[metric_name]}")
        return best_metrics, best_epoch

def gather_metrics_for_experiment_group(experiment_group, average_epochs=0, has_weighted_accuracy=True):
    api = wandb.Api(timeout=60)

    # Fetch runs matching the experiment group
    runs = api.runs("crop-classification/messis", filters={"config.experiment_group": experiment_group})
    print(f"Found {len(runs)} runs for experiment group '{experiment_group}'")

    # Organize the runs by setup name
    setups = {}
    for run in runs:
        if run.state != 'finished':
            print(f"Run {run.url} - Skipping (not finished, state: {run.state})")
            continue
        if 'name' not in run.config:
            if 'experiment_name' in run.config:
                setup_name = run.config['experiment_name'].split('-')[-1]
                print(f"Run {run.url} - Missing 'name' in config, getting it from 'experiment_name' instead: {setup_name}")
            else:
                print(f"Run {run.url} - Missing 'name' in config, skipping")
                continue
        else:
            setup_name = run.config['name']
        if setup_name not in setups:
            setups[setup_name] = {
                'runs': [],
                'metrics': {metric: [] for metric in [
                    'val_f1_tier1_majority',
                    'val_f1_tier2_majority',
                    'val_f1_tier3_majority',
                    'val_weighted_accuracy_tier1_majority' if has_weighted_accuracy else 'val_accuracy_tier1_majority',
                    'val_weighted_accuracy_tier2_majority' if has_weighted_accuracy else 'val_accuracy_tier2_majority',
                    'val_weighted_accuracy_tier3_majority' if has_weighted_accuracy else 'val_accuracy_tier3_majority',
                    'val_precision_tier1_majority',
                    'val_precision_tier2_majority',
                    'val_precision_tier3_majority',
                    'val_recall_tier1_majority',
                    'val_recall_tier2_majority',
                    'val_recall_tier3_majority',
                    'val_cohen_kappa_tier1_majority',
                    'val_cohen_kappa_tier2_majority',
                    'val_cohen_kappa_tier3_majority',
                ]},
                'best_steps': []
            }
        setups[setup_name]['runs'].append(run)

    # Print setup names with number of runs
    for setup_name, setup_data in setups.items():
        print(f"Setup '{setup_name}' - {len(setup_data['runs'])} runs")

    # Gather metrics for each setup
    for setup_name, setup_data in setups.items():
        for run in setup_data['runs']:
            best_metrics, best_epoch = get_best_epoch_metrics(run, 'val_f1_tier3_majority' if setup_name != 'seasonality' else 'val_f1_tier2_majority', average_epochs)
            setup_data['best_steps'].append(best_epoch)
            for metric in setup_data['metrics']:
                # Skip metrics that are not present in the best_metrics
                if metric not in best_metrics:
                    print(f"Run {run.url} - Skipping metric '{metric}' (not present in run)")
                    continue
                setup_data['metrics'][metric].append(best_metrics[metric])
    
    # Prepare data for DataFrame
    data = []
    for setup_name, setup_data in setups.items():
        metrics_summary = {
            'setup': setup_name,
            'num_runs': len(setup_data['runs']),
            'average_epochs': average_epochs,
            'run_handles': [run.id for run in setup_data['runs']],
            'best_steps': setup_data['best_steps'],
        }
        for metric, values in setup_data['metrics'].items():
            metrics_summary[f'{metric}_mean'] = np.mean(values)
            metrics_summary[f'{metric}_std'] = np.std(values)
        data.append(metrics_summary)
    
    metrics_df = pd.DataFrame(data)
    
    return metrics_df

# Run the function for a given experiment group
metrics_df = gather_metrics_for_experiment_group(EXPERIMENT_GROUP, average_epochs=AVERAGE_EPOCHS, has_weighted_accuracy=HAS_WEIGHTED_ACCURACY)

# Save the DataFrame to a CSV file
metrics_df.to_csv(f"experiment_results/eval_{EXPERIMENT_GROUP}.csv", index=False)

metrics_df

Found 20 runs for experiment group 'exp-4-backbone'
Setup 'not_pretrained-vles' - 5 runs
Setup 'not_pretrained' - 5 runs
Setup 'frozen' - 5 runs
Setup 'unfrozen' - 5 runs
Run https://wandb.ai/crop-classification/messis/runs/omulcd0u - Best epoch: 989 - val_f1_tier3_majority: 0.07441633939743042
Run https://wandb.ai/crop-classification/messis/runs/z8t0kbkx - Best epoch: 2107 - val_f1_tier3_majority: 0.09419189393520355
Run https://wandb.ai/crop-classification/messis/runs/jah5taof - Best epoch: 2852 - val_f1_tier3_majority: 0.09031200408935547
Run https://wandb.ai/crop-classification/messis/runs/iq41ryda - Best epoch: 1250 - val_f1_tier3_majority: 0.08324874937534332
Run https://wandb.ai/crop-classification/messis/runs/4vyp456q - Best epoch: 2032 - val_f1_tier3_majority: 0.15356236696243286
Run https://wandb.ai/crop-classification/messis/runs/gg5cwfdb - Best epoch: 1883 - val_f1_tier3_majority: 0.09768390655517578
Run https://wandb.ai/crop-classification/messis/runs/yadnz91s - Best epoch

/Users/yvokeller/Library/Caches/pypoetry/virtualenvs/messis-bV8Bs8aq-py3.12/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/yvokeller/Library/Caches/pypoetry/virtualenvs/messis-bV8Bs8aq-py3.12/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/yvokeller/Library/Caches/pypoetry/virtualenvs/messis-bV8Bs8aq-py3.12/lib/python3.12/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/yvokeller/Library/Caches/pypoetry/virtualenvs/messis-bV8Bs8aq-py3.12/lib/python3.12/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Users/yvokeller/Library/Caches/pypoetry/virtualenvs/messis-bV8B

,setup,num_runs,average_epochs,run_handles,best_steps,val_f1_tier1_majority_mean,val_f1_tier1_majority_std,val_f1_tier2_majority_mean,val_f1_tier2_majority_std,val_f1_tier3_majority_mean,...,val_recall_tier2_majority_mean,val_recall_tier2_majority_std,val_recall_tier3_majority_mean,val_recall_tier3_majority_std,val_cohen_kappa_tier1_majority_mean,val_cohen_kappa_tier1_majority_std,val_cohen_kappa_tier2_majority_mean,val_cohen_kappa_tier2_majority_std,val_cohen_kappa_tier3_majority_mean,val_cohen_kappa_tier3_majority_std
0,not_pretrained-vles,5,0,"[omulcd0u, z8t0kbkx, jah5taof, iq41ryda, 4vyp4...","[989, 2107, 2852, 1250, 2032]",0.367177,0.060494,0.218346,0.035379,0.099146,...,0.221195,0.040429,0.104961,0.032839,0.709980,0.061824,0.579820,0.062790,0.519891,0.063090
1,not_pretrained,5,0,"[gg5cwfdb, yadnz91s, gqr90k2t, 3l8xwfbq, cafhr...","[1883, 3932, 1138, 170, 1995]",0.345397,0.071254,0.193708,0.053979,0.087930,...,0.194557,0.050352,0.092374,0.029086,0.643062,0.159516,0.510248,0.156738,0.456796,0.147686
2,frozen,5,0,"[gute45u0, xn0ssmi9, cr4lsixs, pcifa7t0, q6x22...","[2144, 3038, 1362, 3113, 1064]",0.485859,0.060315,0.289240,0.014321,0.167559,...,0.294730,0.022573,0.170413,0.004435,0.847963,0.010842,0.685353,0.018388,0.635053,0.017838
3,unfrozen,5,0,"[f69quohi, v5mpaa1t, q8hnjw5f, 3vkcbvbk, aoau7...","[2070, 2479, 2256, 1921, 2926]",0.469939,0.069448,0.285159,0.014741,0.183922,...,0.280513,0.023155,0.180178,0.010630,0.860629,0.004269,0.712921,0.007508,0.653462,0.015329


In [11]:
# Mapping for short metric names
metric_name_mapping = {
    'val_f1_tier1_majority': 'F1 $T_1$',
    'val_f1_tier2_majority': 'F1 $T_2$',
    'val_f1_tier3_majority': 'F1 $T_3$',
}

if HAS_WEIGHTED_ACCURACY:
    metric_name_mapping['val_weighted_accuracy_tier1_majority'] = 'W. Acc. $T_1$'
    metric_name_mapping['val_weighted_accuracy_tier2_majority'] = 'W. Acc. $T_2$'
    metric_name_mapping['val_weighted_accuracy_tier3_majority'] = 'W. Acc. $T_3$'
else:
    metric_name_mapping['val_accuracy_tier1_majority'] = 'Acc. $T_1$'
    metric_name_mapping['val_accuracy_tier2_majority'] = 'Acc. $T_2$'
    metric_name_mapping['val_accuracy_tier3_majority'] = 'Acc. $T_3$'

metric_name_mapping = { # Adding further metrics down here, to make sure we keep the most relevant metrics at the top
    **metric_name_mapping,
    'val_precision_tier1_majority': 'Precision $T_1$',
    'val_precision_tier2_majority': 'Precision $T_2$',
    'val_precision_tier3_majority': 'Precision $T_3$',
    'val_recall_tier1_majority': 'Recall $T_1$',
    'val_recall_tier2_majority': 'Recall $T_2$',
    'val_recall_tier3_majority': 'Recall $T_3$',
    'val_cohen_kappa_tier1_majority': 'Kappa $T_1$',
    'val_cohen_kappa_tier2_majority': 'Kappa $T_2$',
    'val_cohen_kappa_tier3_majority': 'Kappa $T_3$',
}

# Function to create a LaTeX table with bold best scores and underlined second-best scores
def create_latex_table(df, baseline_setup, first_n_metrics=None):
    metrics = list(metric_name_mapping.keys())
    if first_n_metrics:
        metrics = metrics[:first_n_metrics]
    
    # Determine the best and second-best scores for each metric
    best_scores = {metric: df[f'{metric}_mean'].max() for metric in metrics}
    second_best_scores = {
        metric: sorted(df[f'{metric}_mean'].unique(), reverse=True)[1] if len(df[f'{metric}_mean'].unique()) > 1 else None
        for metric in metrics
    }

    # Reorder the dataframe to have the baseline setup first
    df = df.set_index('setup')
    if baseline_setup in df.index:
        df = df.loc[[baseline_setup] + [idx for idx in df.index if idx != baseline_setup]].reset_index()
    else:
        df = df.reset_index()

    # Initialize the Texttable object
    table = tt.Texttable()
    
    # Define the header
    headers = ['Metric'] + [f"{'$\\dagger$ ' if row['setup'] == baseline_setup else ''}{str(setup).replace('_', '\\_')} ({row['num_runs']}-fold)" 
                            for idx, row in df.iterrows() for setup in [row['setup']]]
    table.header(headers)
    
    # Set alignment for columns
    cols_align = ["c"] * len(headers)
    cols_align[0] = "l"
    table.set_cols_align(cols_align)
    
    # Populate the table with data
    for metric in metrics:
        row_data = [metric_name_mapping[metric]]
        for idx, row in df.iterrows():
            mean = row[f'{metric}_mean'] * 100
            std = row[f'{metric}_std'] * 100
            value = f'{mean:.1f}\\% ± {std:.1f}\\%'
            if mean == best_scores[metric] * 100:
                value = f'\\textbf{{{value}}}'
            elif second_best_scores[metric] and mean == second_best_scores[metric] * 100:
                value = f'\\underline{{{value}}}'
            row_data.append(value)
        table.add_row(row_data)
    
    # Generate the LaTeX table
    average_epochs_used = df['average_epochs'].iloc[0] 
    caption_averaged =  "" if average_epochs_used == 0 else f" averaged over {average_epochs_used} epochs"
    latex_table = latextable.draw_latex(
        table, 
        caption=f"Results for experiment {EXPERIMENT_GROUP}. All metrics calculated on field majority during validation phase{caption_averaged}, with mean and std reported. Best scores bold, second best underlined, $\\dagger$ is baseline.", 
        label=f"tab:metrics_comparison_{EXPERIMENT_GROUP}", 
        use_booktabs=True,
        position="h"
    )

    # Add \resizebox to the LaTeX table
    latex_table = latex_table.replace('\\begin{center}', '\\begin{center}\\resizebox{\\textwidth}{!}{')
    latex_table = latex_table.replace('\\end{center}', '}\\end{center}')

    return latex_table

# Generate the LaTeX table
df = pd.read_csv(f"experiment_results/eval_{EXPERIMENT_GROUP}.csv")
df = df.sort_values(by='setup')

# Optional: Drop rows to exclude some setups
df = df.drop(EXCLUDED_SETUPS)

latex_table = create_latex_table(df, baseline_setup=BASELINE_SETUP, first_n_metrics=FIRST_N_METRICS)
print(latex_table)

\begin{table}[h]
	\begin{center}\resizebox{\textwidth}{!}{
		\begin{tabular}{lcc}
			\toprule
			Metric & $\dagger$ full-neck (5-fold) & bottleneck-rf-4 (5-fold) \\
			\midrule
			F1 $T_1$ & \underline{41.4\% ± 4.6\%} & \textbf{47.3\% ± 6.8\%} \\
			F1 $T_2$ & \underline{28.8\% ± 1.1\%} & \textbf{30.4\% ± 2.3\%} \\
			F1 $T_3$ & \textbf{19.3\% ± 1.0\%} & \underline{18.9\% ± 0.5\%} \\
			W. Acc. $T_1$ & \textbf{93.1\% ± 0.3\%} & \underline{92.5\% ± 0.4\%} \\
			W. Acc. $T_2$ & \textbf{82.1\% ± 0.9\%} & \underline{80.8\% ± 1.5\%} \\
			W. Acc. $T_3$ & \textbf{77.1\% ± 1.3\%} & \underline{76.2\% ± 1.3\%} \\
			\bottomrule
		\end{tabular}
	}\end{center}
	\caption{Results for experiment exp-13-neck-reduction. All metrics calculated on field majority during validation phase, with mean and std reported. Best scores bold, second best underlined, $\dagger$ is baseline.}
	\label{tab:metrics_comparison_exp-13-neck-reduction}
\end{table}


In [14]:
# deserialize best_steps column in df to list
df['best_steps'] = df['best_steps'].apply(eval)

In [23]:
# calculate mean and std for each list in column best_steps
df['best_steps_mean'] = df['best_steps'].apply(np.mean)
df['best_steps_std'] = df['best_steps'].apply(np.std)

In [24]:
# show columns setup, best_steps_mean and num_runs
df[['setup', 'best_steps_mean', 'best_steps_std', 'num_runs']]

,setup,best_steps_mean,best_steps_std,num_runs
2,frozen,2144.2,838.584617,5
1,not_pretrained,1823.6,1239.572281,5
3,unfrozen,2330.4,351.503400,5
